# Colab 量化分析工作流程

In [ ]:
# 儲存格 1: (可選) 掛載 Google Drive
# 執行此儲存格可以將您的分析結果永久保存在 Google Drive。
# 如果跳過，所有成果將在 Colab 關閉後刪除。
from google.colab import drive
import os

#@markdown ### **設定 Google Drive 專案目錄**
#@markdown 請輸入您希望在 Google Drive 中使用的專案根目錄名稱。
DRIVE_PROJECT_ROOT = "MyQuantProject" #@param {type:"string"}

try:
    print("正在嘗試掛載 Google Drive...")
    drive.mount('/content/drive', force_remount=True)
    gdrive_path = os.path.join('/content/drive/MyDrive', DRIVE_PROJECT_ROOT)
    os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path
    os.environ['PROJECT_ROOT_PATH'] = gdrive_path
    print(f"✅ Google Drive 掛載成功！專案雲端路徑設定為：{gdrive_path}")

    for folder in ['data', 'models', 'results']:
        folder_path = os.path.join(gdrive_path, folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"📁 已在雲端建立資料夾：{folder_path}")
except Exception as e:
    os.environ['GDRIVE_PROJECT_PATH'] = ""
    print(f"⚠️ 掛載 Google Drive 失敗或未授權。錯誤：{e}")
    print("後續儲存格將使用 Colab 本機儲存空間。")

In [ ]:
# 儲存格 2: 從 GitHub 同步專案程式碼
# 從指定的 GitHub 儲存庫和分支下載最新的程式碼。
import os

#@markdown ### **1. 設定 GitHub 儲存庫**
GITHUB_REPO_URL = "https://github.com/hsp1234-web/qlib.git" #@param {type:"string"}

#@markdown ---
#@markdown ### **2. 🚀 輸入要執行的分支名稱**
GIT_BRANCH = "" #@param {type:"string"}

if not GIT_BRANCH or GIT_BRANCH.strip() == "":
    raise ValueError("❌ 錯誤：分支名稱 (GIT_BRANCH) 不得為空！請在上方欄位中輸入您要執行的分支。")

os.environ['GITHUB_REPO_URL'] = GITHUB_REPO_URL
os.environ['GIT_BRANCH'] = GIT_BRANCH.strip()

repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
local_repo_path = os.path.join('/content', repo_name)

print(f"目標 GitHub 儲存庫：{GITHUB_REPO_URL}")
print(f"目標分支：{os.environ['GIT_BRANCH']}")
print(f"預計本地路徑：{local_repo_path}")

if os.path.exists(local_repo_path):
    print(f"\n專案資料夾已存在，正在切換至 '{os.environ['GIT_BRANCH']}' 分支並更新...")
    os.chdir(local_repo_path)
    get_ipython().system(f'git fetch origin && git checkout {os.environ["GIT_BRANCH"]} && git pull origin {os.environ["GIT_BRANCH"]}')
    os.chdir('/content')
else:
    print(f"\n專案資料夾不存在，正在從 '{os.environ['GIT_BRANCH']}' 分支下載整個專案...")
    get_ipython().system(f'git clone --branch {os.environ["GIT_BRANCH"]} {GITHUB_REPO_URL}')

print("\n✅ 程式碼同步完成！")

In [ ]:
# 儲存格 3: 安裝專案所需的 Python 套件
# 讀取 requirements.txt 檔案，並安裝所有指定的依賴項。
import os

GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')
if not GITHUB_REPO_URL:
    raise ValueError("❌ 錯誤：找不到 GITHUB_REPO_URL 環境變數。請先執行儲存格 2。")

repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
requirements_path = os.path.join('/content', repo_name, 'requirements.txt')

if os.path.exists(requirements_path):
    print(f"正在從 {requirements_path} 安裝依賴套件...")
    get_ipython().system(f'pip install -q -r {requirements_path}')
    print("\n✅ 環境安裝完成！")
else:
    print(f"⚠️ 警告：在 '{requirements_path}' 中找不到 requirements.txt 檔案。")

In [ ]:
# 儲存格 4: 執行核心任務：下載數據與執行回測
# 根據下方設定的參數，執行資料下載 (data_loader.py) 與回測分析 (run_analysis.py)。
import os

GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')
if not GITHUB_REPO_URL:
    raise ValueError("❌ 錯誤：找不到 GITHUB_REPO_URL 環境變數。請先執行儲存格 2。")

#@markdown ### **回測參數設定**
STOCK_ID = "2330.TW" #@param {type:"string"}
START_DATE = "2020-01-01" #@param {type:"string"}
END_DATE = "2023-12-31" #@param {type:"string"}

project_root = os.environ.get('PROJECT_ROOT_PATH')
if not project_root:
    print("⚠️ 提醒：您未執行儲存格 1 或 Google Drive 掛載失敗。")
    print("將自動在本機建立臨時專案目錄。執行結果將在 Colab 階段作業結束後遺失。")
    local_project_name = "MyQuantProject_local"
    project_root = os.path.join('/content', local_project_name)
    os.environ['PROJECT_ROOT_PATH'] = project_root
    print(f"📁 本機專案路徑已設定為：{project_root}")

os.makedirs(os.path.join(project_root, 'data'), exist_ok=True)
os.makedirs(os.path.join(project_root, 'results'), exist_ok=True)

repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
scripts_path = f"/content/{repo_name}/src"
data_output_path = os.path.join(project_root, 'data', f'{STOCK_ID}.csv')
results_output_path = os.path.join(project_root, 'results', 'backtest_results.json')

print("\n--- 步驟 1: 執行 data_loader.py ---")
data_loader_cmd = f"python {scripts_path}/data_loader.py --stock_id '{STOCK_ID}' --start_date '{START_DATE}' --end_date '{END_DATE}' --output_path '{data_output_path}'"
get_ipython().system(data_loader_cmd)

print("\n--- 步驟 2: 執行 run_analysis.py ---")
run_analysis_cmd = f"python {scripts_path}/run_analysis.py --data_path '{data_output_path}' --results_path '{results_output_path}'"
get_ipython().system(run_analysis_cmd)

print("\n✅ 核心任務執行完畢！")
print(f"📈 結果已儲存至：{results_output_path}")

In [ ]:
# 儲存格 5: 啟動 Streamlit 視覺化儀表板
# 將上一步的回測結果，透過一個公開的網址，以互動式儀表板呈現。
import os
import subprocess
import time

print("正在安裝 ngrok...")
get_ipython().system('pip install -q pyngrok')
from pyngrok import ngrok

GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')
if not GITHUB_REPO_URL: raise ValueError("❌ 錯誤：找不到 GITHUB_REPO_URL。請先執行儲存格 2。")

project_root = os.environ.get('PROJECT_ROOT_PATH')
if not project_root: raise ValueError("❌ 錯誤：找不到 PROJECT_ROOT_PATH。請先執行儲存格 4。")

repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
dashboard_script_path = f"/content/{repo_name}/src/dashboard.py"
results_json_path = os.path.join(project_root, 'results', 'backtest_results.json')

if not os.path.exists(dashboard_script_path): raise FileNotFoundError(f"❌ 錯誤：找不到儀表板腳本 '{dashboard_script_path}'。")
if not os.path.exists(results_json_path): raise FileNotFoundError(f"❌ 錯誤：找不到結果檔案 '{results_json_path}'。")

command = f"streamlit run {dashboard_script_path} -- --results_path='{results_json_path}' --server.port 8501 --server.headless true"

print("\n正在背景啟動 Streamlit 服務...")
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)

ngrok.kill()
public_url = ngrok.connect(8501)
print("✅ Streamlit 儀表板已啟動！")
print(f"👉 請點擊此公開網址查看：{public_url}")

In [ ]:
# 儲存格 6: (可選) 同步本地成果至 Google Drive
# 如果您在未掛載雲端的情況下執行了分析，此儲存格可將本地成果同步回您的 Google Drive。
import os
from google.colab import drive
from distutils.dir_util import copy_tree

gdrive_path = os.environ.get('GDRIVE_PROJECT_PATH')
project_root = os.environ.get('PROJECT_ROOT_PATH')

if not project_root or not os.path.exists(project_root):
    print("❌ 錯誤：找不到任何本地專案成果。請先執行儲存格 4。")
elif gdrive_path and project_root == gdrive_path:
    print("✅ 您的專案成果已經儲存在 Google Drive 中，無需同步。")
else:
    print("🔄 開始將本地成果同步至 Google Drive...")
    if not gdrive_path:
        try:
            print("正在嘗試掛載 Google Drive...")
            drive.mount('/content/drive', force_remount=True)
            project_name = "MyQuantProject"
            gdrive_path = os.path.join('/content/drive/MyDrive', project_name)
            os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path
            print(f"✅ Google Drive 掛載成功！雲端路徑設定為：{gdrive_path}")
        except Exception as e:
            print(f"❌ 掛載 Google Drive 失敗。無法同步。錯誤：{e}")
            gdrive_path = None
    if gdrive_path:
        print(f"\n正在將 '{project_root}' 的內容複製到 '{gdrive_path}'...")
        os.makedirs(gdrive_path, exist_ok=True)
        copy_tree(project_root, gdrive_path)
        print("✅ 同步完成！所有本地成果已成功複製到 Google Drive。")